In [2]:
import ee
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [1]:
import pandas as pd
import numpy as np
import matplotlib
import scipy
import folium
import shapely
from IPython.display import Image

In [3]:
import geopandas as gpd
import rasterio
import pygeohydro

In [ ]:
meadows_df = gpd.read_parquet("./meadows_data/ucdmeadows/ucdmeadows.parquet")
meadows_df.head()
meadows_df = meadows_df.to_crs("WGS84")

In [ ]:
## get all the data and layers that intersect with points in one huc12
sample = meadows_df.loc[meadows_df['HUC12'] == '180300020601']
sample

In [ ]:
wbd = pygeohydro.watershed.WBD('huc12')
data = wbd.byids(field='huc12', fids=['180201210605'])
data = data.to_crs(meadows_df.crs)
data.iloc[0].geometry
data.crs


1.    get landsat that intersects with data geometry where data is WBD return union month=month
2.   get monthly average for all bands in month mosaiced
3.   add bands for elevation, chili, mTPI, slope
4.   make feature collection from meadows
5.   split into train/test
6.   train classifier
7.   get metrics

In [ ]:
import geemap.foliumap as geemap

In [ ]:
from geemap.plot import geopandas_to_ee
study_area = geopandas_to_ee(data)

In [ ]:
def bitwise_extract(input, start=3, end=4):
        mask_size = ee.Number(1).add(end).subtract(start)
        mask = ee.Number(1).leftShift(mask_size).subtract(1)
        return input.rightShift(start).bitwiseAnd(mask)

def mask_clouds(image):
    # https://spatialthoughts.com/2021/08/19/qa-bands-bitmasks-gee/
    # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2#bands

    qa = image.select('QA_PIXEL')
    # clouds are Bit 3, cloud shadow is Bit 4
    cloud_mask = bitwise_extract(qa).eq(0)
    image = image.updateMask(cloud_mask)
    return image

def scale_factor(image):
    optical = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    image = image.addBands(optical, None, True)
    image = image.addBands(thermal, None, True)
    return image

def compute_indices(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename("NDVI")
    image = image.addBands(ndvi)
    return image

def clip_sa(ic, study_area):
    def clip(image):
      return image.clip(study_area)
    return ic.map(clip)

ls =  ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate('2013-05-01', '2013-09-30')
ls = ls.filterBounds(study_area)
ls = clip_sa(ls, study_area)
ls = ls.map(scale_factor)
ls = ls.map(mask_clouds)
ls = ls.map(compute_indices)

In [ ]:
Map = geemap.Map(zoom=4, crs="EPSG4326")
Map.set_center(-150.7, 21.7)
first = ls.first()
vis_params = {'bands':['SR_B4', 'SR_B3', 'SR_B2'], 'min':0, 'max':0.2}
ndvi_vis_params = {'bands': ['NDVI'], 'min':-1, 'max':1}
Map.addLayer(first, vis_params, 'landsat')
Map.addLayer(first, ndvi_vis_params, 'ndvi')
Map

In [ ]:
Map = geemap.Map(zoom=4, crs="EPSG4326")
Map.set_center(-150.7, 21.7)
land_cover = ee.ImageCollection('USGS/NLCD_RELEASES/2019_REL/NLCD')
land_cover = land_cover.filter(ee.Filter.eq('system:index', '2019')).first()
land_cover = land_cover.select('landcover')
Map.addLayer(land_cover, None, 'Landcover')
Map

In [ ]:
Map = geemap.Map(zoom=4, crs="EPSG4326")
Map.set_center(-150.7, 21.7)
dem = ee.Image('USGS/SRTMGL1_003');
elevation = dem.select('elevation');
slope = ee.Terrain.slope(elevation);
Map.addLayer(slope, {'min': 0, 'max': 60}, 'slope');
Map.addLayer(elevation, {'min': 0, 'max':3000}, 'elevation');
# add flow accumulation
# add TPI
# add chili
# add aspect
# add curvature
Map

In [ ]:
# tpi

In [ ]:
Map = geemap.Map(zoom=4, crs="EPSG4326")
Map.set_center(-150.7, 21.7)
naip = ee.ImageCollection('USDA/NAIP/DOQQ')
naip = naip.filter(ee.Filter.date('2019-06-01', '2020-07-01'))
truecolor = naip.select(['R', 'G', 'B']);
vis_params = {
  min: 0,
  max: 255,
};
Map.addLayer(truecolor, vis_params, 'True Color');
Map

In [ ]:
wbd = pygeohydro.watershed.WBD('huc12')
data = wbd.byids(field='huc12', fids=['180400090505'])
#meadows_df['huc10'] = meadows_df['HUC12'].apply(lambda x: x[:-2])
meadows = meadows_df.loc[meadows_df['HUC12'].isin(['180400090505'])]
study_area_ee = geopandas_to_ee(data)
meadows_ee = geopandas_to_ee(meadows)

In [ ]:
# Option 1:
# given a set of points (some meadow, some non-meadow)
# use Image.sampleRegions on the points to get the feature values
# train cart classifier

Map = geemap.Map(zoom=4, crs="EPSG4326")
Map.set_center(-149.9, 20.5)
naip = ee.ImageCollection('USDA/NAIP/DOQQ')
naip = naip.filter(ee.Filter.date('2018-05-01', '2020-07-01'))
naip = naip.filterBounds(study_area.geometry())
naip = naip.median()
naip = naip.clip(study_area_ee.geometry())
truecolor = naip.select(['R', 'G', 'B']);
vis_params = {
  'min': 0,
  'max': 255,
  'gamma': [1.4, 1.4, 1.4]
};
Map.addLayer(truecolor, vis_params, 'True Color')
Map.addLayer(meadows_ee, {'opacity': 0.5, 'color': 'green'}, 'meadows')
Map

In [ ]:
meadows